# Load data

In [ ]:
import json

with open ("人工客服设置SOP.json", 'r+') as f:
    chunks1 = []
    for i in json.load(f):
        chunks1.append({'text':i["text"].strip("。") + '。'})
with open ("工单管理.json", 'r+') as f:
    chunks2 = []
    for i in json.load(f):
        chunks2.append({'text':i["text"].strip("。") + '。'})

# Similarity

install https://github.com/rango-ramesh/advanced-chunker


replace core.py in SemanticChunker

In [ ]:
from semantic_chunker.core import SemanticChunker
import matplotlib.pyplot as plt

chunker1 = SemanticChunker(model_name="sentence-transformers/bge-m3", max_tokens=8192, cluster_threshold=0.5, method=False)
# all-MiniLM-L6-v2 | m3e-large | m3e-base | bge-m3
merged_chunks = chunker1.chunk(chunks1)

In [ ]:
for i, merged in enumerate(merged_chunks):
    print(f"Chunk {i}:")
    print(merged["text"])
    print()
plt.plot(chunker1.get_similarity())
plt.show()

In [ ]:
from semantic_chunker.core import SemanticChunker
import matplotlib.pyplot as plt

chunker2 = SemanticChunker(model_name="sentence-transformers/bge-m3", max_tokens=8192, cluster_threshold=0.8, method=True)
# all-MiniLM-L6-v2 | m3e-large | m3e-base | bge-m3
merged_chunks = chunker2.chunk(chunks2)

In [ ]:
for i, merged in enumerate(merged_chunks):
    print(f"Chunk {i}:")
    print(merged["text"])
    print()
plt.plot(chunker2.get_similarity())
plt.show()

# BERT

In [ ]:
class Chunk_timestamps:
    def __init__(self,
                 line = 0,
                 chunk=None,
                 start_time=None,
                 end_time=None):
        self.line = line
        self.chunk = chunk
        self.start_time = start_time
        self.end_time = end_time

    def getstamps(self):
        return {
                "end_time": self.end_time,
                "line": self.line,
                "text": self.chunk,
                "start_time": self.start_time
            }

### Sentence Match

install modelscope

In [ ]:
import json

with open ("人工客服设置SOP.json", 'r+') as f:
    chunks1 = []
    for i in json.load(f):
        chunks1.append({'text':i["text"].strip("。") + '。'})
with open ("工单管理.json", 'r+') as f:
    chunks2 = []
    for i in json.load(f):
        chunks2.append({'text':i["text"].strip("。") + '。'})

In [ ]:
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline


documents1 = ""
for i in chunks1:
    documents1 += i["text"]

p = pipeline(
    task=Tasks.document_segmentation,
    model='modelscope/Seqmodel', model_revision='master')

result = p(documents1)

In [ ]:
chunks = [chunk.strip("\t") for chunk in result[OutputKeys.TEXT].split("\n") if len(chunk.strip("\t")) > 0]
import json

with open("工单管理.json", "r", encoding="utf-8") as f:
    data = json.load(f)

chunk_timestamps = []
line = 0
for chunk in chunks:
    newchunk = Chunk_timestamps(chunk=chunk, line=line)
    found1 = False
    found2 = False
    for entry in data:
        if chunk.split("。")[0] in entry["text"]:
            newchunk.start_time = entry["start_time"]
            found1 = True
        if chunk.split("。")[-2] in entry["text"]:
            newchunk.end_time = entry["end_time"]
            found2 = True
        if found1 and found2:
            found1 = False
            found1 = False
            break
    if not found1 and not found2:
        chunk_timestamps.append({
            "end_time": None,
            "text": chunk,
            "start_time": None
        })
    chunk_timestamps.append(newchunk)
    line += 1

for item in chunk_timestamps:
    print(item.getstamps())

### Word Match

In [ ]:
import json

with open ("response2.json", 'r+') as f:
    chunks1 = []
    for i in json.load(f):
        chunks1.append({'text':i["text"].strip("。") + '。'})

with open("response2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline


documents1 = ""
for i in chunks1:
    documents1 += i["text"]

p = pipeline(
    task=Tasks.document_segmentation,
    model='modelscope/Seqmodel', model_revision='master')

result = p(documents1)

In [ ]:
chunks = [chunk.strip("\t") for chunk in result[OutputKeys.TEXT].split("\n") if len(chunk.strip("\t")) > 0]
chunks

In [ ]:
def seconds_to_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int(round((seconds - int(seconds)) * 1000))
    return f"{hours:02}:{minutes:02}:{secs:02}.{millis:03}"

In [ ]:
chunk_timestamps = []
line = 0
for chunk in chunks:
    newchunk = Chunk_timestamps(chunk=chunk, line=line)
    found1 = False
    found2 = False
    for entry in data:
        if chunk.split("。")[0] in entry["text"]:
            for word in entry["words"]:
                if chunk.split("。")[0][0] == word["word"][0]:
                    newchunk.start_time = seconds_to_srt_time(word["start"])
                    found1 = True
        if chunk.split("。")[-2] in entry["text"]:
            for word in entry["words"]:
                if chunk.split("。")[-2][-1] == word["word"][0]:
                    newchunk.end_time = seconds_to_srt_time(word["end"])
                    found2 = True
        if found1 and found2:
            found1 = False
            found1 = False
            break
    if not found1 and not found2:
        chunk_timestamps.append({
            "end_time": None,
            "text": chunk,
            "start_time": None
        })
    chunk_timestamps.append(newchunk)
    line += 1

for item in chunk_timestamps:
    print(item.getstamps())

In [ ]:
import json
with open("results/result1.json", "w+", encoding="utf-8") as f:
    json.dump([item.getstamps() for item in chunk_timestamps], f, ensure_ascii=False, indent=2)

In [ ]:
with open("results/result1.json", "r+", encoding="utf-8") as f:
    infos = json.load(f)

In [ ]:
import subprocess
cnt = 0
video = "test/62beb8064262ea272ccb6e94cfc7fb6e.mp4"
for info in infos:
    start_time = info["start_time"]
    end_time = info["end_time"]
    output = "test/result" + str(cnt) + ".mp4"
    ffmpeg_command1 = "D:/desktop/1/work/ai4c/stt/ffmpeg -y -i " +  video
    ffmpeg_command2= " -ss " + start_time
    ffmpeg_command3 = " -to " + end_time + " -c copy"
    ffmpeg_command4 = " -vcodec libx264 -acodec libmp3lame " + output
    ffmpeg_command = ffmpeg_command1 + ffmpeg_command2 + ffmpeg_command3 + ffmpeg_command4
    print(ffmpeg_command)
    p = subprocess.Popen(ffmpeg_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    print(out.decode(), err.decode())
    cnt += 1

# Late Chunking

install transformers

In [ ]:
import json

with open ("人工客服设置SOP.json", 'r+') as f:
    chunks1 = []
    for i in json.load(f):
        chunks1.append({'text':i["text"].strip("。") + '。'})
with open ("工单管理.json", 'r+') as f:
    chunks2 = []
    for i in json.load(f):
        chunks2.append({'text':i["text"].strip("。") + '。'})

documents1 = ""
for i in chunks1:
    documents1 += i["text"]

documents2 = ""
for i in chunks2:
    documents2 += i["text"]

In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/jina-embeddings-v2-base-zh', trust_remote_code=True)
model = AutoModel.from_pretrained('sentence-transformers/jina-embeddings-v2-base-zh', trust_remote_code=True)

In [ ]:
def chunk_by_sentences(input_text: str, tokenizer,  split_token = "。"):
    """文本切块+找到每个文本块在token粒度的索引范围"""
    print("input_text:", input_text)
    inputs = tokenizer(input_text, return_tensors='pt', return_offsets_mapping=True)
    punctuation_mark_id = tokenizer.convert_tokens_to_ids(split_token)
    token_offsets = inputs['offset_mapping'][0]
    # 保证最后的句子保存起来
    sep_id = int(token_offsets[-1][0])
    token_ids = inputs['input_ids'][0]
    # 找到文本粒度的划分
    chunk_positions = []
    for i, (token_id, (start, end)) in enumerate(zip(token_ids, token_offsets)):
        if token_id == punctuation_mark_id:
            if token_offsets[i + 1][0] - token_offsets[i][1] >= 0 or token_offsets[i + 1][0]==sep_id:
                chunk_positions.append((i, int(start + 1))) 
    chunks = [
        input_text[x[1] : y[1]]
        for x, y in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    span_annotations = [
        (x[0], y[0]) for (x, y) in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    return chunks, span_annotations


split_token = "。"
chunks, span_annotations = chunk_by_sentences(documents2, tokenizer, split_token=split_token) 
print('Chunks:\n- "' + '"\n- "'.join(chunks) + '"')
print(span_annotations)

In [ ]:
def chunked_pooling(
    model_output, span_annotation: list, max_length=None
):
    """对token embedding序列分chunk做mean pooling"""
    token_embeddings = model_output[0]
    outputs = []
    for embeddings, annotations in zip(token_embeddings, span_annotation):
        if (
            max_length is not None
        ):  # remove annotations which go bejond the max-length of the model
            annotations = [
                (start, min(end, max_length - 1))
                for (start, end) in annotations
                if start < (max_length - 1)
            ]
        pooled_embeddings = [
            embeddings[start:end].sum(dim=0) / (end - start)
            for start, end in annotations
            if (end - start) >= 1
        ]
        pooled_embeddings = [
            embedding.detach().cpu().numpy() for embedding in pooled_embeddings
        ]
        outputs.append(pooled_embeddings)

    return outputs


inputs = tokenizer(documents2, return_tensors='pt')
model_output = model(**inputs)
embeddings = chunked_pooling(model_output, [span_annotations])[0]
embeddings_traditional_chunking = model.encode(chunks)

In [ ]:
import numpy as np

threshold = 0.96
cos_sim = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
new_chunks = []
new_chunk = [chunks[0]]
new_similarities = []
similarities = []
length = len(embeddings) - 1
i = 0
while i < length:
    new_similarities.append(cos_sim(embeddings[i], embeddings[i+1]))
    similarities.append(cos_sim(embeddings_traditional_chunking[i], embeddings_traditional_chunking[i+1]))
    if cos_sim(embeddings[i], embeddings[i+1]) > threshold:
        new_chunk.append(chunks[i+1])
        i += 1
    else:
        new_chunks.append(new_chunk)
        new_chunk = [chunks[i+1]]
        i += 1
if len(new_chunk) >= 1:
    new_chunks.append(new_chunk)
print(len(new_chunks))
new_chunks

In [ ]:
similarities

In [ ]:
from matplotlib import pyplot as plt
plt.plot(similarities, 'b-')
plt.show()
plt.plot(new_similarities, 'r-')
plt.show()